In [3]:
import pandas as pd

# Reload uploaded files
transactions_df = pd.read_csv(r"C:\Users\vigne\OneDrive\Desktop\TRANSACTION_TAKEHOME.csv")
products_df = pd.read_csv(r"C:\Users\vigne\OneDrive\Desktop\PRODUCTS_TAKEHOME.csv")
users_df = pd.read_csv(r"C:\Users\vigne\OneDrive\Desktop\USER_TAKEHOME.csv")

## Part 1: Data Exploration

### Objective
Review the provided CSV files to identify:
- Data quality issues
- Fields that may be difficult to interpret or analyze
This is done using Python (Pandas) and data visualization tools.

---

### Files Analyzed
- `USER_TAKEHOME.csv` – user demographic and profile data
- `TRANSACTION_TAKEHOME.csv` – scanned receipts and transactional data
- `PRODUCTS_TAKEHOME.csv` – product catalog and brand/category metadata

---

### Are there any data quality issues present?

#### PRODUCTS_TAKEHOME.csv
| Column        | Null Count  | Issue                                 |
|---------------|-------------|----------------------------------------|
| CATEGORY_4    | 778,093      | Over 90% missing → possibly deprecated |
| CATEGORY_3    | 60,566       | Moderate nulls, check usefulness       |
| MANUFACTURER  | 226,474      | May impact brand-level analysis        |

#### TRANSACTION_TAKEHOME.csv
- `BARCODE` values may appear in scientific notation (e.g., `7.83E+11`) → breaks joins
- `FINAL_QUANTITY` includes the string `"zero"` instead of numeric `0`
- `FINAL_SALE` contains blank strings (`" "`) → causes numeric conversion issues

#### USER_TAKEHOME.csv
- `GENDER` contains inconsistent formats like `"male"`, `"Male"`, `"Non-Binary"`, `"prefer_not_to_say"`
- Some missing `BIRTH_DATE` values → affects age-based segmentation

---

### Are there any fields that are challenging to understand?

- **`CATEGORY_4`**: Sparsely populated and likely outdated
- **`BARCODE`**: Appears in different formats across files; needs normalization
- **`FINAL_QUANTITY`**: Stored as string instead of numeric; must be cleaned
- **`GENDER`**: Requires normalization for consistent grouping
- **`SCAN_DATE` vs `PURCHASE_DATE`**: Their business difference is unclear without documentation

---

### Recommendations

| Issue                        | Suggested Fix                                                                 |
|-----------------------------|-------------------------------------------------------------------------------|
| Missing `CATEGORY_4`        | Drop or ignore for analysis                                                  |
| `BARCODE` in sci notation   | Convert to text or integer using `'{:.0f}'.format()` in Python               |
| `"zero"` in `FINAL_QUANTITY`| Replace with `'0'`, cast to numeric                                           |
| Blank `FINAL_SALE`          | Use `NULLIF(final_sale, '')::NUMERIC` in SQL                                 |
| `GENDER` inconsistencies    | Normalize to: `Male`, `Female`, `Non-Binary`, `Other`, `Prefer Not to Say`   |
| `SCAN_DATE` meaning unclear | Confirm with stakeholders for proper usage                                   |


In [4]:
# Display the first few rows of each dataframe
users_df.head(), transactions_df.head(), products_df.head()

(                         ID               CREATED_DATE  \
 0  5ef3b4f17053ab141787697d  2020-06-24 20:17:54.000 Z   
 1  5ff220d383fcfc12622b96bc  2021-01-03 19:53:55.000 Z   
 2  6477950aa55bb77a0e27ee10  2023-05-31 18:42:18.000 Z   
 3  658a306e99b40f103b63ccf8  2023-12-26 01:46:22.000 Z   
 4  653cf5d6a225ea102b7ecdc2  2023-10-28 11:51:50.000 Z   
 
                   BIRTH_DATE STATE LANGUAGE  GENDER  
 0  2000-08-11 00:00:00.000 Z    CA   es-419  female  
 1  2001-09-24 04:00:00.000 Z    PA       en  female  
 2  1994-10-28 00:00:00.000 Z    FL   es-419  female  
 3                        NaN    NC       en     NaN  
 4  1972-03-19 00:00:00.000 Z    PA       en  female  ,
                              RECEIPT_ID PURCHASE_DATE  \
 0  0000d256-4041-4a3e-adc4-5623fb6e0c99    2024-08-21   
 1  0001455d-7a92-4a7b-a1d2-c747af1c8fd3    2024-07-20   
 2  00017e0a-7851-42fb-bfab-0baa96e23586    2024-08-18   
 3  000239aa-3478-453d-801e-66a82e39c8af    2024-06-18   
 4  00026b4c-dfe8-49dd-

In [5]:
# Step 2: Identify and summarize data quality issues across datasets

# Null counts
users_nulls = users_df.isnull().sum()
transactions_nulls = transactions_df.isnull().sum()
products_nulls = products_df.isnull().sum()

# Unique values in questionable fields
gender_values = users_df['GENDER'].dropna().unique()
quantity_values = transactions_df['FINAL_QUANTITY'].dropna().unique()
barcode_types = transactions_df['BARCODE'].dropna().apply(lambda x: type(x)).value_counts()
final_sale_blank = (transactions_df['FINAL_SALE'].astype(str).str.strip() == '').sum()

# Count missing birth dates
missing_birthdates = users_df['BIRTH_DATE'].isnull().sum()

# Prepare summary
summary_issues = {
    "Users - Missing GENDER values": users_nulls['GENDER'],
    "Users - Missing BIRTH_DATE": missing_birthdates,
    "Users - GENDER variations": list(gender_values),
    "Transactions - FINAL_QUANTITY values": list(quantity_values),
    "Transactions - BARCODE data types": barcode_types.to_dict(),
    "Transactions - Blank FINAL_SALE": final_sale_blank,
    "Products - Null values": products_nulls[products_nulls > 0].to_dict()
}

summary_issues


{'Users - Missing GENDER values': np.int64(5892),
 'Users - Missing BIRTH_DATE': np.int64(3675),
 'Users - GENDER variations': ['female',
  'male',
  'non_binary',
  'transgender',
  'prefer_not_to_say',
  'not_listed',
  'Non-Binary',
  'unknown',
  'not_specified',
  "My gender isn't listed",
  'Prefer not to say'],
 'Transactions - FINAL_QUANTITY values': ['1.00',
  'zero',
  '2.00',
  '3.00',
  '4.00',
  '4.55',
  '2.83',
  '2.34',
  '0.46',
  '7.00',
  '18.00',
  '12.00',
  '5.00',
  '2.17',
  '0.23',
  '8.00',
  '1.35',
  '0.09',
  '2.58',
  '1.47',
  '16.00',
  '0.62',
  '1.24',
  '1.40',
  '0.51',
  '0.53',
  '1.69',
  '6.00',
  '2.39',
  '2.60',
  '10.00',
  '0.86',
  '1.54',
  '1.88',
  '2.93',
  '1.28',
  '0.65',
  '2.89',
  '1.44',
  '2.75',
  '1.81',
  '276.00',
  '0.87',
  '2.10',
  '3.33',
  '2.54',
  '2.20',
  '1.93',
  '1.34',
  '1.13',
  '2.19',
  '0.83',
  '2.61',
  '0.28',
  '1.50',
  '0.97',
  '0.24',
  '1.18',
  '6.22',
  '1.22',
  '1.23',
  '2.57',
  '1.07',
  '2

In [6]:
# Normalize the gender field in users_df
import numpy as np

# Define a mapping for normalization
gender_mapping = {
    'male': 'Male',
    'female': 'Female',
    'non_binary': 'Non-Binary',
    'non-binary': 'Non-Binary',
    'transgender': 'Other',
    'not_specified': 'Prefer Not to Say',
    'prefer_not_to_say': 'Prefer Not to Say',
    'not_listed': 'Prefer Not to Say',
    'unknown': 'Prefer Not to Say',
    "my gender isn't listed": 'Prefer Not to Say',
    'prefer not to say': 'Prefer Not to Say'
}

# Convert to lowercase, strip, and map
users_df['GENDER'] = users_df['GENDER'].astype(str).str.strip().str.lower().map(gender_mapping)

# Count the cleaned gender values
gender_clean_counts = users_df['GENDER'].value_counts(dropna=False)

gender_clean_counts


GENDER
Female               64240
Male                 25829
NaN                   5892
Other                 1772
Prefer Not to Say     1760
Non-Binary             507
Name: count, dtype: int64

In [7]:
# Fix FINAL_QUANTITY and FINAL_SALE fields in transactions_df

# Replace 'zero' (case-insensitive) with '0' in FINAL_QUANTITY
transactions_df['FINAL_QUANTITY'] = transactions_df['FINAL_QUANTITY'].replace(
    to_replace=r'(?i)\bzero\b', value='0', regex=True)

# Convert FINAL_QUANTITY to numeric (if possible)
transactions_df['FINAL_QUANTITY'] = pd.to_numeric(transactions_df['FINAL_QUANTITY'], errors='coerce')

# Replace blank strings and whitespace in FINAL_SALE with NaN
transactions_df['FINAL_SALE'] = transactions_df['FINAL_SALE'].replace(r'^\s*$', np.nan, regex=True)

# Convert FINAL_SALE to numeric
transactions_df['FINAL_SALE'] = pd.to_numeric(transactions_df['FINAL_SALE'], errors='coerce')

# Summary after cleaning
quantity_summary = transactions_df['FINAL_QUANTITY'].describe()
sale_null_count = transactions_df['FINAL_SALE'].isnull().sum()

quantity_summary, sale_null_count


(count    50000.000000
 mean         0.814427
 std          1.841794
 min          0.000000
 25%          0.007500
 50%          1.000000
 75%          1.000000
 max        276.000000
 Name: FINAL_QUANTITY, dtype: float64,
 np.int64(12500))

In [8]:
transactions_df.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.0,NaN
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,0.0,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.0,NaN
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,0.0,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.0,NaN


In [9]:
# Fix BARCODE formatting in both transactions and products datasets

# Convert barcodes to numeric (if not already), then to string with no scientific notation
transactions_df['BARCODE'] = pd.to_numeric(transactions_df['BARCODE'], errors='coerce')
transactions_df['BARCODE'] = transactions_df['BARCODE'].apply(lambda x: '{:.0f}'.format(x) if pd.notnull(x) else x)

products_df['BARCODE'] = pd.to_numeric(products_df['BARCODE'], errors='coerce')
products_df['BARCODE'] = products_df['BARCODE'].apply(lambda x: '{:.0f}'.format(x) if pd.notnull(x) else x)

# Check sample output
barcode_samples = {
    "Transactions Sample": transactions_df['BARCODE'].dropna().unique()[:5].tolist(),
    "Products Sample": products_df['BARCODE'].dropna().unique()[:5].tolist()
}

barcode_samples


{'Transactions Sample': ['15300014978',
  '78742229751',
  '783399746536',
  '47900501183',
  '681131411295'],
 'Products Sample': ['796494407820',
  '23278011028',
  '461817824225',
  '35000466815',
  '806810850459']}

## Further Data Quality Issues and What Can Be Done

After cleaning gender, quantity, sale values, and barcode formatting, we ran additional checks and found the following issues:

---

### 1. Duplicate Barcodes in Product Data
- 185 barcodes are repeated in `PRODUCTS_TAKEHOME.csv`.
- This can cause problems when joining transactions to product metadata.

**What can be done:**
- Keep only the first (or most complete) record for each barcode using `.drop_duplicates(subset='BARCODE')`.

---

### 2. Quantity Outliers (Greater than 100)
- Found 2 transactions where `FINAL_QUANTITY` > 100.
- These might be bulk purchases or data entry mistakes.

**What can be done:**
- Flag these as outliers and exclude them from average quantity or distribution plots.

---

### 3. FINAL_QUANTITY = 0 but FINAL_SALE > 0
- Over 12,000 rows have 0 quantity but positive sale values.
- This can distort per-unit or average item calculations.

**What can be done:**
- Exclude these records when computing unit-based metrics.
- Keep them in revenue-only summaries (if sale amount is reliable).

---

### 4. USER_ID Not Found in Users Table
- 49,738 transactions have `USER_ID`s that do not match any ID in the users table.
- These records cannot be linked to a known user profile.

**What can be done:**
- Remove these rows before doing user-level analysis (like age, gender, state).
- Optionally group them as "unidentified users" if overall transaction count matters.


In [28]:
# Apply filters and corrections based on further data issues

# 1. Remove duplicate barcodes from products
products_cleaned = products_df.drop_duplicates(subset='BARCODE')

# 2. Remove outliers with FINAL_QUANTITY > 100
transactions_no_outliers = transactions_df[transactions_df['FINAL_QUANTITY'] <= 100]

# 3. Remove transactions where FINAL_QUANTITY = 0 but FINAL_SALE > 0
transactions_filtered = transactions_no_outliers[
    ~((transactions_no_outliers['FINAL_QUANTITY'] == 0) & (transactions_no_outliers['FINAL_SALE'] > 0))
]

# 4. Keep only transactions with valid users
valid_user_ids = set(users_df['ID'])
transactions_final = transactions_filtered[transactions_filtered['USER_ID'].isin(valid_user_ids)]

# Final cleaned transaction and product datasets
final_transaction_count = len(transactions_final)
final_product_count = len(products_cleaned)

final_transaction_count, final_product_count


(190, 841343)